In [22]:
#Step1. Data Stock : Crawling Naver "https://finance.naver.com/"
# - BrautifulSoup - Request - Dataframe
#어떠한 정보를 Gathering 할 것 인가. 
# 거래량, 외인 거래량, 기관 거래량, 시가(전날 종가가 시가인데 의미가있나?), 종가 ?
#Step2. Data Gathering : CSV, DB?
# - 
#Step3. Model
# - 분류를 어떻게 나눌것인가 (Classifier : 종가 상승인지 하강인지? or 5분 뒤 상승인지 하강인지?)
# - Model : Random Forest?

In [23]:
# 불필요한 경고 메시지 끄기
import warnings
warnings.filterwarnings('ignore')

In [24]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from matplotlib import font_manager
import requests
from openpyxl import Workbook
from io import BytesIO
from bs4 import BeautifulSoup
import re

In [25]:
custom_headers = {
    "referer":'https://naver.com/',
    "user-agent":'Mozilla'
}

In [26]:
wb = Workbook()
ws = wb.active
ws.append(["일자","종가","거래량","기관순매수","기관순매매","보유율"])
code = input("종목 코드 : ")
max_page = input("수집할 페이지 수 : ")

종목 코드 : 034730
수집할 페이지 수 : 100


In [27]:
if max_page:
    max_page = int(max_page)
else:
    max_page = 0

In [28]:
data = []

In [29]:
for myPage in range(1,max_page):
    url = f"https://finance.naver.com/item/frgn.naver?code={code}&page={myPage}"
    res = requests.get(url, headers=custom_headers)
    soup = BeautifulSoup(res.text,"lxml")
    pageTable = soup.find("table", attrs={ "class" : "Nnavi", "summary":"페이지 네비게이션 리스트"})
    if int(pageTable.find("td",attrs = {"class":"on"}).text.strip()) != myPage:
        break
    table = soup.find("table", attrs={ "class" : "type2", "summary":"외국인 기관 순매매 거래량에 관한표이며 날짜별로 정보를 제공합니다."})
    for idx, dataraw in enumerate(table.findAll("tr")):
        if idx <= 2:
            continue
        cols = dataraw.findAll("td")
        cols = [ele.text.strip() for ele in cols]
        data.append([ele for ele in cols if ele]) 
    

In [30]:
df = pd.DataFrame(data)

In [52]:
dscolumnls = ["날짜","종가","전일비","등락률","거래량","순매매량","순매매량","외인보유주수","외인보유율"]

In [55]:
df.columns = dscolumnls

In [56]:
df = df.dropna()

In [35]:
df.to_csv('SK_StockData.csv', sep=',', na_rep='NaN')

In [36]:
#Data Fram중에 필요한 정보를 Collection 한다 
#Data Frame 의 모든 String 을 Int or Date 등등으로 변경한다.
#그래프도 한번 그려보자
#포인트도 짝어보자
#모델은 뭐쓰지?

In [37]:
df['날짜'] = pd.to_datetime(df['날짜'])

In [39]:
df.set_index('날짜', inplace=True)

In [57]:
dfordered = df

In [58]:
dfordered

,종가,전일비,등락률,거래량,순매매량,순매매량,외인보유주수,외인보유율
날짜,,,,,,,,
2022-03-11,"247,000","10,500",+4.44%,"304,312","+138,876","-61,186","15,968,341",21.54%
2022-03-10,"236,500","5,500",+2.38%,"197,274","+83,292","-48,073","16,029,527",21.62%
2022-03-08,"231,000","5,000",+2.21%,"202,304","+89,380","-36,308","16,076,016",21.68%
2022-03-07,"226,000","3,500",-1.53%,"157,996","+5,087","-17,156","16,112,324",21.73%
2022-03-04,"229,500","3,000",-1.29%,"116,835","-18,770","-4,623","16,135,380",21.76%
...,...,...,...,...,...,...,...,...
2014-02-26,"138,000","2,000",-1.43%,"52,606","+1,683","+3,251","9,601,889",19.20%
2014-02-25,"140,000",500,+0.36%,"102,559",-462,"-8,539","9,597,698",19.20%
2014-02-24,"139,500","3,500",+2.57%,"154,101","+66,370","-38,312","9,606,237",19.21%


In [59]:
#df 정보 확인
dfordered.describe()

,종가,전일비,등락률,거래량,순매매량,순매매량,외인보유주수,외인보유율
count,1980,1980,1980,1980,1980,1980,1980,1980
unique,349,49,706,1971,1969,1951,1980,671
top,"270,000","1,000",0.00%,"186,213","+13,894","+6,922","15,968,341",27.62%
freq,24,202,114,2,2,3,1,13


In [60]:
#df 누락 여부 확인, object 항목들에 data 화 필요
#전날 대비 종가 상승인지 하락인지 확인하는 추가 Column 필요?????? 그게 맞나?
#5,10,20,30일선 추가
#전일비, 등락률 Column 삭제
dfordered.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1980 entries, 2022-03-11 to 2014-02-20
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   종가      1980 non-null   object
 1   전일비     1980 non-null   object
 2   등락률     1980 non-null   object
 3   거래량     1980 non-null   object
 4   순매매량    1980 non-null   object
 5   순매매량    1980 non-null   object
 6   외인보유주수  1980 non-null   object
 7   외인보유율   1980 non-null   object
dtypes: object(8)
memory usage: 139.2+ KB


In [62]:
dfordered.drop(["전일비","등락률","외인보유주수"],axis="columns",inplace = True)

In [67]:
dfordered

,종가,거래량,순매매량,순매매량,외인보유율
날짜,,,,,
2022-03-11,"247,000","304,312","+138,876","-61,186",21.54%
2022-03-10,"236,500","197,274","+83,292","-48,073",21.62%
2022-03-08,"231,000","202,304","+89,380","-36,308",21.68%
2022-03-07,"226,000","157,996","+5,087","-17,156",21.73%
2022-03-04,"229,500","116,835","-18,770","-4,623",21.76%
...,...,...,...,...,...
2014-02-26,"138,000","52,606","+1,683","+3,251",19.20%
2014-02-25,"140,000","102,559",-462,"-8,539",19.20%
2014-02-24,"139,500","154,101","+66,370","-38,312",19.21%


In [68]:
dfordered["종가"]

날짜
2022-03-11    247,000
2022-03-10    236,500
2022-03-08    231,000
2022-03-07    226,000
2022-03-04    229,500
               ...   
2014-02-26    138,000
2014-02-25    140,000
2014-02-24    139,500
2014-02-21    136,000
2014-02-20    135,500
Name: 종가, Length: 1980, dtype: object

chan test